# Convert RNMC SQLite files to SPPARKS format
Converts both initial_state.sqlite and reaction_network.sqlite


In [34]:
import sqlite3
import numpy as np
import pandas as pd

In [35]:
def sqlite_to_df(path, column_name):
    '''Take in path to sqlite file and return data frame of specified columns'''
    
    # Read sqlite query results into a pandas DataFrame
    con = sqlite3.connect(path)
    df = pd.read_sql_query("SELECT * FROM " + column_name, con)
    con.close()
    
    return df   

In [54]:
########################### USER_DEFINED ##############################
path_state = '../../LGMC/SEI/initial_state.sqlite'
path_rn = '../../LGMC/SEI/rn.sqlite'
file_path_input = 'input_small.txt'
#######################################################################

df_initial_state = sqlite_to_df(path_state, 'initial_state')
df_rn = sqlite_to_df(path_rn, 'reactions')

species = df_initial_state['species_id'].to_numpy()

In [61]:
df_rn

,reaction_id,number_of_reactants,number_of_products,reactant_1,reactant_2,product_1,product_2,phase_reactant_1,phase_reactant_2,phase_product_1,phase_product_2,dG,prefactor,rate,electron_tunneling_coefficient,reorganization_energy,charge_transfer_coefficient,type
0,0,1,1,4,-1,5,-1,S,S,S,S,-1.210000,1.0,1.076125e+06,0.0,0.000000,0.0,S
1,1,1,1,5,-1,4,-1,S,S,S,S,1.210000,1.0,3.789910e-15,0.0,0.000000,0.0,S
2,2,2,2,5,5,9,12,S,S,S,S,-2.680000,1.0,6.212438e+12,0.0,0.000000,0.0,S
3,3,2,2,9,12,5,5,S,S,S,S,2.680000,1.0,3.103738e-33,0.0,0.000000,0.0,S
4,4,1,2,6,-1,12,7,S,S,S,S,-1.610000,1.0,3.942822e+10,0.0,0.000000,0.0,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,94,1,1,3,-1,4,-1,S,N,S,N,-2.169667,1000000.0,0.000000e+00,1.2,1.451307,0.5,R
95,95,1,1,5,-1,6,-1,L,N,L,N,-2.896727,1000000.0,0.000000e+00,1.2,0.927523,0.5,R
96,96,1,1,5,-1,6,-1,S,N,S,N,-2.896727,1000000.0,0.000000e+00,1.2,0.927523,0.5,R
97,97,1,1,13,-1,14,-1,L,N,L,N,-1.436019,1000000.0,0.000000e+00,1.2,1.117988,0.5,R


In [55]:
# create mapping between numerical species of alphabetical names
# mapping = {i : chr(int(i/26) + 65) + chr(i%26 + 65) for i in range(np.shape(species)[0])}
mapping = {i : chr(int(i/26) + 65) + chr(i%26 + 65) for i in range(15)}

In [56]:
# create file with basic chemistry commands

f = open(file_path_input, 'w')

f.write('# SPPARKS chemistry application \n')
f.write('# RNMC example \n')
f.write('seed 1234 \n')
f.write('app_style chemistry\n')
f.write('solve_style tree \n')
f.write('volume 1.9098395e-11\n')


21

In [57]:
# add species

f = open(file_path_input, 'a')

for i in mapping:
    f.write('add_species ' + mapping[i] + '\n')

In [58]:
# add reactions

f = open(file_path_input, 'a')
for index, row in df_rn.iterrows():
    
    f.write('add_reaction ' + str(index) + ' ')
    
    if(row['number_of_reactants'] == 1):
        f.write(mapping[row['reactant_1']] + ' ')
    else:
        f.write(mapping[row['reactant_1']] + ' ' + mapping[row['reactant_2']] + ' ')
    
    f.write(str(row['rate']) + ' ')
        
    if(row['number_of_products'] == 1):
        f.write(mapping[row['product_1']] + '\n')
    else:
        f.write(mapping[row['product_1']] + ' ' + mapping[row['product_2']] + '\n')

In [59]:
# initial state
f = open(file_path_input, 'a')

for index, row in df_initial_state[df_initial_state['count'] > 0].iterrows():
    f.write('count ' + mapping[row['species_id']] + ' ' + str(row['count']) + '\n')
    
f.close()

In [60]:
for index, row in df_rn.iterrows():
    
    print('add_reaction ' + str(index), end=' ')
    
    if(row['number_of_reactants'] == 1):
        print(mapping[row['reactant_1']], end=' ')
    else:
        print(mapping[row['reactant_1']] + ' ' + mapping[row['reactant_2']], end=' ')
    
    print(str(row['rate']), end=' ')
        
    if(row['number_of_products'] == 1):
        print(mapping[row['product_1']])
    else:
        print(mapping[row['product_1']] + ' ' + mapping[row['product_2']])

add_reaction 0 AE 1076124.5088206409 AF
add_reaction 1 AF 3.789910034483802e-15 AE
add_reaction 2 AF AF 6212437992903.239 AJ AM
add_reaction 3 AJ AM 3.103738471444452e-33 AF AF
add_reaction 4 AG 39428223988.561714 AM AH
add_reaction 5 AM AH 2.405326570773392e-17 AG
add_reaction 6 AH AD 24218231.791125935 AJ
add_reaction 7 AJ 1.526578694329515e-19 AH AD
add_reaction 8 AH AB 21953.948267463245 AL
add_reaction 9 AL 2.842548598390442 AH AB
add_reaction 10 AO 660.9943374139933 AI AF
add_reaction 11 AI AF 2.5592323733916926e-27 AO
add_reaction 12 AE 1076124.5088206409 AF
add_reaction 13 AF 3.789910034483802e-15 AE
add_reaction 14 AF AF 6212437992903.239 AJ AM
add_reaction 15 AJ AM 3.103738471444452e-33 AF AF
add_reaction 16 AG AA 39428223988.561714 AM AH
add_reaction 17 AM AH 2.405326570773392e-17 AG AA
add_reaction 18 AH AD 24218231.791125935 AJ AA
add_reaction 19 AJ AA 1.526578694329515e-19 AH AD
add_reaction 20 AH AB 21953.948267463245 AL AA
add_reaction 21 AL AA 2.842548598390442 AH AB
a